## Aquí realizaremos el ETL al archivo "australian_users_items.json"

Importaremos las librerias principales

In [1]:
import json
import pandas as pd
import ast
import numpy as np

Impotaremos el archivo 

In [12]:
ruta = "australian_users_items.json"

# Inicializamo una lista vacía para almacenar las filas del archivo JSON
fila = []    
with open(ruta, encoding="utf-8") as f:
    for line in f.readlines():
        fila.append(ast.literal_eval(line)) #evalúa expresiones literales de Python de una cadena de texto de manera segura.

# Cargamos los datos en un DataFrame utilizando la lista de filas
df_items = pd.DataFrame(fila)
# Mostramos las primeras 8 líneas del Dataframe
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Este código desanida una columna que contiene listas, reinicia los índices y luego fusiona el DataFrame original con la columna desanidada basándose en el índice.

In [13]:
# Desanidar la columna 'items' utilizando el método explode()
df_items_desanidado = df_items['items'].explode()

# Reiniciar el índice en ambos DataFrames
df_items_reset = df_items.reset_index()
df_items_desanidado_reset = df_items_desanidado.reset_index()

# Fusionar los DataFrames basándote en el índice
df_merge = df_items_reset.merge(df_items_desanidado_reset, left_on='index', right_on='index')

La columna user_url no aporta ningún valor a nuestro proyecto y la columna item_x es el resultado del merge entre los datasets df_items_reset y df_items_desanidado_reset la cual es redundante con la columna items_y la cual permanecerá y le cambiaremos el nombre a items.

In [14]:
# Eliminar las columnas index, user_url e items_x
df_user_items = df_merge.drop(['user_url', 'items_x'], axis=1)

# Cambiar el nombre de la columna reviews_x a reviews
df_user_items = df_user_items.rename(columns={'items_y': 'items'})

Procederemos a desanidar la columna items

In [15]:
# Convierte la columna 'reviews' en un DataFrame anidado
items_normalized = pd.json_normalize(df_user_items['items'])

# Combina el nuevo DataFrame con el original
df_user_items = pd.concat([df_user_items, items_normalized], axis=1)

# Elimina la columna original 'reviews'
df_user_items = df_user_items.drop('items', axis=1)

# Muestra el DataFrame resultante
df_user_items

,index,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,0,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,0,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,0,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170010,88308,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,88308,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,88308,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5170013,88308,76561198329548331,7,76561198329548331,519140,Minds Eyes,3.0,3.0


En nuestro proyecto es de vital importancia el espacio en memoria, por eso en nuestro enfoque vamos a elimimnar valores nulos y cualquier otro tipo de dato inconsistente para tener un dataset lo mas ligero posible. en las siguientes dos líneas de código eliminaremos los juegos con que preseten 0 horas de juego y cualquier dato nulo presente.

In [16]:
df_user_items = df_user_items[df_user_items['playtime_forever'] != 0.0]
df_user_items = df_user_items.dropna()

In [17]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5170013
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   index             int64  
 1   user_id           object 
 2   items_count       int64  
 3   steam_id          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 225.6+ MB


Procederemos a eliminar las columnas item_count, steam_id, playtime_2weeks ya que no aportan a nuestro proyecto y nos ayuda a hacer mas ligero el dataset

In [18]:
df_user_items = df_user_items.drop(['items_count', 'steam_id', 'playtime_2weeks'], axis=1)
df_user_items.head(5)

,index,user_id,item_id,item_name,playtime_forever
0,0,76561197970982479,10,Counter-Strike,6.0
2,0,76561197970982479,30,Day of Defeat,7.0
8,0,76561197970982479,300,Day of Defeat: Source,4733.0
9,0,76561197970982479,240,Counter-Strike: Source,1853.0
10,0,76561197970982479,3830,Psychonauts,333.0


La columna playtime_forever almacena el tiempo de juego en minutos, la transformaremos para que muestre el tiempo jugado en horas.

In [19]:
# Dividir la columna playtime_forever por 60 para convertir minutos a horas
df_user_items['playtime_forever'] = df_user_items['playtime_forever'] / 60

In [20]:
# Redondear la columna playtime_forever a un solo decimal
df_user_items['playtime_forever'] = df_user_items['playtime_forever'].round(1)

In [21]:
df_user_items

,index,user_id,item_id,item_name,playtime_forever
0,0,76561197970982479,10,Counter-Strike,0.1
2,0,76561197970982479,30,Day of Defeat,0.1
8,0,76561197970982479,300,Day of Defeat: Source,78.9
9,0,76561197970982479,240,Counter-Strike: Source,30.9
10,0,76561197970982479,3830,Psychonauts,5.6
...,...,...,...,...,...
5170007,88308,76561198329548331,304930,Unturned,11.3
5170008,88308,76561198329548331,227940,Heroes & Generals,0.7
5170011,88308,76561198329548331,388490,One Way To Die: Steam Edition,0.0
5170012,88308,76561198329548331,521570,You Have 10 Seconds 2,0.1


exportaremos el Dataframe en formato csv.

In [22]:
df_user_items.to_csv('./user_items.csv', index=False)

Hemos tomado del dataset "australian_users_items.json" y tomamos las características que consideramos más importantes, en nuestro EDA procederemos con un analisis mas profundo.